In [2]:
from openpyxl import load_workbook
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string, get_column_letter
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

from filtering import *
from loading import *

PATHS = ['SAMPLES\zz_lista_volontera_22_23.xlsx', 'SAMPLES\zz_lista_volontera_23_24.xlsx', 'SAMPLES\zz_lista_volontera_24_25.xlsx']
LAST_DATES = ["13.4.2023.", "13.4.2024.", "13.4.2025."]

dfs = []

for i in range(len(PATHS)):

    year_df = load_year(PATHS[i])
    year_df = remove_non_wanted_dates(year_df, LAST_DATES[i])
        
    year_df = year_df.sort_values(by='volunteer_dates', key=lambda x: x.apply(len), ascending=False)
    year_df = year_df.reset_index(drop=True)
        
    year_df['volunteer_count'] = year_df['volunteer_dates'].apply(len)
    
    dfs.append(year_df)
    
display(dfs[2].head(10))

<>:11: SyntaxWarning: invalid escape sequence '\z'
<>:11: SyntaxWarning: invalid escape sequence '\z'
<>:11: SyntaxWarning: invalid escape sequence '\z'
<>:11: SyntaxWarning: invalid escape sequence '\z'
<>:11: SyntaxWarning: invalid escape sequence '\z'
<>:11: SyntaxWarning: invalid escape sequence '\z'
C:\Users\Lobel\AppData\Local\Temp\ipykernel_30108\865537450.py:11: SyntaxWarning: invalid escape sequence '\z'
  PATHS = ['SAMPLES\zz_lista_volontera_22_23.xlsx', 'SAMPLES\zz_lista_volontera_23_24.xlsx', 'SAMPLES\zz_lista_volontera_24_25.xlsx']
C:\Users\Lobel\AppData\Local\Temp\ipykernel_30108\865537450.py:11: SyntaxWarning: invalid escape sequence '\z'
  PATHS = ['SAMPLES\zz_lista_volontera_22_23.xlsx', 'SAMPLES\zz_lista_volontera_23_24.xlsx', 'SAMPLES\zz_lista_volontera_24_25.xlsx']
C:\Users\Lobel\AppData\Local\Temp\ipykernel_30108\865537450.py:11: SyntaxWarning: invalid escape sequence '\z'
  PATHS = ['SAMPLES\zz_lista_volontera_22_23.xlsx', 'SAMPLES\zz_lista_volontera_23_24.xlsx', 

Loading volunteers for P...
Loading volunteers for T...
Loading volunteers for P...
Loading volunteers for J...
Loading volunteers for T...
Loading volunteers for P...
Loading volunteers for Š...
Loading volunteers for K...


,volunteer_name,volunteer_dates,volunteer_class,volunteer_school,volunteer_count
0,Jakov Kifer,"[(Trešnjevka, 1.10.2024.), (Trešnjevka, 3.10.2...",2. razred,PŠVP,50
1,Borna Planinić,"[(Trešnjevka, 1.10.2024.), (Trešnjevka, 3.10.2...",2. razred,PŠVP,46
2,Rita Bakić,"[(Trešnjevka, 19.11.2024.), (Peščenica, 1.10.2...",3. razred,XV.,44
3,Lobel Marunić,"[(Trešnjevka, 1.10.2024.), (Trešnjevka, 8.10.2...",3. razred,XV.,33
4,Sven Biban,"[(Trešnjevka, 8.10.2024.), (Trešnjevka, 22.10....",3. razred,KG,31
5,Juran Carev,"[(Peščenica, 10.10.2024.), (Peščenica, 15.10.2...",3. razred,XV.,29
6,Lucija Ćurić,"[(Peščenica, 1.10.2024.), (Peščenica, 8.10.202...",3. razred,XV.,27
7,Ema Donev,"[(Peščenica, 1.10.2024.), (Peščenica, 3.10.202...",3. razred,XV.,21
8,Karla Ćosić,"[(Peščenica, 1.10.2024.), (Peščenica, 3.10.202...",2. razred,XV.,20
9,Juraj Šegon,"[(Peščenica, 3.10.2024.), (Peščenica, 10.10.20...",4. razred,XV.,19


In [ ]:
# Per location, average attendance

prijavljeni_volonteri = pd.DataFrame({
    'Lokacija': ['Peščenica', 'Trešnjevka', 'Špansko', 'Kralj Tomislav'],
    'Prosječna dolaznost': [11.21, 4.24, 5.91, 5.87]
})
prijavljeni_volonteri = prijavljeni_volonteri.sort_values(by='Prosječna dolaznost', ascending=True)

plt.figure(figsize=(6, 6), dpi=800)
plt.bar(prijavljeni_volonteri['Lokacija'], prijavljeni_volonteri['Prosječna dolaznost'], color=['skyblue', 'orange', 'green'])

# Add title and labels
plt.title('Prosječna dolaznost volontera, 24./25.', fontsize=16, weight='bold', pad=20)
plt.xlabel('Prosječna dolaznost', fontsize=14, labelpad=15)
plt.ylabel('Lokacija', fontsize=14, labelpad=15)

# Adjust layout to make room for labels
plt.tight_layout()

# Save the figure with 800 DPI before showing it
fig = plt.gcf()
fig.savefig('./REZULTATI/4_mjesec/prosjecna_dolaznost_24_25.png', dpi=800)

# Display the plot
plt.show()

In [ ]:
# For getting the total number of volunteers at a specific location

df = dfs[2]

# Flatten the 'dates' column
df_flattened = df.explode('volunteer_dates')

# Split the 'dates' tuple into separate columns (location and date)
df_flattened[['location', 'date']] = pd.DataFrame(df_flattened['volunteer_dates'].to_list(), index=df_flattened.index)

# Filter by a specific location (e.g., 'Location 1')
location_to_check = 'Peščenica'
df_flattened = df_flattened[df_flattened['location'] == location_to_check]

df_flattened['date'] = df_flattened['volunteer_dates'].apply(lambda x: x[1] if isinstance(x, tuple) and len(x) > 1 else None)
unique_dates = df_flattened['date'].nunique()

print(f"Number of volunteers at {location_to_check}: {unique_dates}")

In [ ]:
# Percentage of non-active volunteers

year_df = dfs[2]

non_active = year_df[year_df['volunteer_count'] == 0]

percentage_non_active = len(non_active) / len(year_df) * 100

print(len(year_df) - len(non_active))

# Plot the percentage of non-active volunteers
plt.figure(figsize=(6, 6), dpi=800)
plt.pie([100 - percentage_non_active, percentage_non_active], labels=['Aktivni', 'Neaktivni'], autopct='%1.1f%%', colors=['lightblue', 'lightcoral'], startangle=90)

# Add title
plt.title('Neaktivni, 14.4., 2024./2025.', fontsize=20, weight='bold', pad=20)
plt.tight_layout()
# Save the figure with 800 DPI before showing it
fig = plt.gcf()
fig.savefig('./REZULTATI/4_mjesec/neaktivnih/percentage_non_active_volunteers-24_25.png', dpi=800)

# Display the plot
plt.show()

In [ ]:
# Number of hours per school
year_df = dfs[0]


school_hours = year_df.groupby('volunteer_school')['volunteer_count'].sum().reset_index()
school_hours = school_hours.sort_values(by='volunteer_count', ascending=False)
school_hours.columns = ['Škola', 'Broj sati']
school_hours = school_hours.reset_index(drop=True)

# Assuming you already have 'school_hours' DataFrame
plt.figure(figsize=(10, 6), dpi=800)  # Set the DPI to 800 for both display and saving
plt.bar(school_hours['Škola'], school_hours['Broj sati'], color=['skyblue', 'orange', 'green', 'red', 'purple'])

# Add title and labels
plt.title('Broj sati po školama, 2022./2023.', fontsize=16, weight='bold', pad=20)
plt.xlabel('Škola', fontsize=14, labelpad=15)
plt.ylabel('Broj sati', fontsize=14, labelpad=15)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Adjust layout to make room for rotated labels
plt.tight_layout()  # Adjust layout to fit labels properly

# Save the figure with 800 DPI before showing it
fig = plt.gcf()  # Get the current figure
fig.savefig('./REZULTATI/4_mjesec/sati_po_skoli/sati_po_skoli-graf-22_23.png', dpi=800)

# Display the plot
plt.show()

print(school_hours)

In [ ]:
# Number of hours at a specific location

year_df = dfs[2]

# Count only the tuples where the first element is 'Trešnjevka'
count = year_df['volunteer_dates'].apply(
    lambda lst: sum(1 for tup in lst if tup[0] == 'Kralj Tomislav')
).sum()

print(count)

49


In [65]:
# Number of hours in a specific school / class

year_df = dfs[2]

filtered_df = year_df[year_df['volunteer_class'] == '4. razred']

count = filtered_df['volunteer_dates'].apply(len).sum()

print(f'Prijava: {len(filtered_df)}')
print(f'Sati: {count}')

Prijava: 33
Sati: 76


In [5]:
# Number of hours by gender

female_names = ['Rita Bakić', 'Lucija Ćurić', 'Ema Donev', 'Karla Ćosić', 'Leona Ozimin Ptiček', 'Nika Milinković', 'Maja Lea Matić Zbiljski', 'Margita Krnić', 'Anja Vukobratović', 'Hana Banjanin', 'Petra Grubeša', 'Lara Paić', 'Viktorija Lozančić ', 'Lara Pavić', 'Maša Leona Lepur', 'Maja Kobešćak', 'Marina Vlahović', 'Anja Kozomara', 'Leda Silov', 'Ema Merslavić', 'Ema Kajba', 'Ana Jukić', 'Ilka Medić', 'Lana Cvok', 'Matea Jenjić', 'Klara Tomšić', 'Una Kozina', 'Kaja Eršek', 'Lorena Perić', 'Leona Vukoja', 'Rosa Bakić', 'Sara Filipčić', 'Ana Brkljačić', 'Ema Čepo', 'Petra Šverko', 'Vita Petrović', 'Lasta Trupčević', 'Ida Bakran Mujkić', 'Lola Jelčić', 'Karla Martinac', 'Klara Bilandžija', 'Lara Brlek', 'Iva Morović', 'Lola Ćurlin', 'Lucija Livaja', 'Laura Paar', 'Franka Mandarić', 'Eva Pudar', 'Petra Mešić', 'Sheena Žmavčić', 'Dea Mračkovski', 'Monika Bodo', 'Nina Bošnjak', 'Paola Gjuriš', 'Sara Polašek', 'Dora Štimac', 'Dora Palata', 'Emanuela Raguž', 'Mia Gudelj', 'Vita Lovrić', 'Korina Malogorski', 'Katarina Adamović', 'Ana Penava', 'Katja Božić', 'Mare Zrno Agoli', 'Lucija Knežević', 'Olja Krajnović', 'Zora Mudrovčić', 'Ema Lovašen', 'Nika Lukenda', 'Sara Hrman', 'Patricia Galić', 'Nika Antunović', 'Ema Todić', 'Helena Štengl', 'Mare Barić', 'Lucija Gabrić', 'Jana Krajnović', 'Lara Vlajčević', 'Klara Gnjidic ', 'Korina Žvigač', 'Anja Kokanović', 'Ana Sekušak', 'Ana Lakoš', 'Lana Šekerija', 'Karla Pogelšek', 'Korina Maren', 'Lana Srića', 'Lana Sulić', 'Kristina Arambašić', 'Klara Kukoč', 'Ema Hrvoić', 'Nina Magzan', 'Petra Cvjetičanin', 'Chiara Tomić', 'Antea Puljak', 'Barbara Tršek', 'Marta Milovac', 'Greta Skočić', 'Marta Bačlija', 'Mia Perlić', 'Mirta Globočnik', 'Lorena Smrekar', 'Lara Radelić', 'Klara Dojčinović', 'Katja Lončarić', 'Sunčica Đurasek', 'Eva Durmiš', 'Nika Franulović', 'Tonka Zujić', 'Karla Perić', 'Karla Peric', 'Lara Džubur Krajnović', 'Lara Novaković', 'Lana Škarica', 'Tonka Ljubić', 'Anja Petrak', 'Andreja Štimec', 'Ana Princip', 'Ana Čović', 'Ana Maršić', 'Ana Jurić', 'Ana Bago', 'Una Tomas Tunjić', 'Valentina Šimičević', 'Vanja Cizel?', 'Vida Kangrga', 'Vita Miholić', 'Jelena Ferenčak', 'Jana Šljerac', 'Jana Vidović', 'Jana Hadžisejdić', 'Jana Krivohvalek', 'Julijana Pavleković', 'Gita Crevar', 'Mara Marta Gadže', 'Inga Jembrek', 'Magdalena Filipaj', 'Magdalena Vrdoljak', 'Magdalena Aničić', 'Karla Gugo', 'Lucija Srzić ', 'Lucija Kršić', 'Lucija Jurišević', 'Lola Prebeg', 'Lucija Vuković', 'Lucija Vrljić', 'Jana Krizmanić', 'Lucija Robić', 'Leda Kefeček', 'Lara Šimić', 'Lea Đigaš', 'Kiara Milavić', 'Klara Botić', 'Katja Vranić', 'Nea Međugorac', 'Mila Krešić', 'Mihael Klarić', 'Mia Ledinšćak', 'Matea Kovilić', 'Marta Maver', 'Marta Žuvić', 'Marija Miloš', 'Marta Bezjak', 'Iris Pašalić', 'Marija Babić', 'Dora Fiket', 'Gabrijela Salic?', 'Gabriela Miloš', 'Dora Posavec', 'Elena Beštak', 'Nika Lukač', 'Nikola Ivančević', 'Franka Ljolje', 'Franka Islamović', 'Erika Gržanić', 'Paula Keros', 'Pavla Žaja', 'Olha Kurinna', 'Paola Gjuris', 'Nika Novaković', 'Nika Osvald', 'Nika Šajnović', 'Rahela Matoković', 'Anna Sukhareva', 'Branka Užarević', 'Petra Hanžel', 'Petra Dokupil', 'Bruna Palinec', 'Sanja Ivkošić', 'Sara Ljevar', 'Sofija Dumančić', 'Tara Bosek', 'Tea Buntak', 'Tea Vodanović', 'Tia Pitra', 'Tin Boršić', 'Zoe Posokhova', 'Zora Hranuelli ']
male_names = ['Jakov Kifer', 'Borna Planinić', 'Lobel Marunić', 'Sven Biban', 'Juran Carev', 'Juraj Šegon', 'Mauro Kritovac', 'Vito Popović', 'Ivo Šimić', 'Ante Kelava', 'Karlo Krstulović ', 'Fran Veselko', 'Noa Majetić', 'Roko Dumanić', 'Mak Štefanec', 'Mate Radoš', 'Matija Vreš', 'Adam Bessen', 'Hrvoje Valent', 'Luka Števanja', 'Luka Šipek', 'Luka Vlah', 'Lovro Smojver', 'Adrian Babić Herceg', 'Sven Ridzak', 'Matej Futivić', 'Leon Rilović', 'Borna Raguž', 'Bruno Brajković', 'Marin Puljiz', 'Filip Knežević', 'Luka Alar', 'Luka Cerruti', 'Andrej Makarović', 'Tomas Klasić', 'Vito Pokupčić', 'Bartol Kranjčević', 'Roko Bošnjak', 'Grgur Lončar', 'Ivan Matić', 'Juraj Miličević', 'Luka Matković', 'Jakov Stjepanović', 'Luka Adžić', 'Luka Mikulić', 'Leon Baždarić', 'Rio Uroda', 'Noah Podnar', 'Fran Markus', 'Filip Krznar', 'Fran Knežević', 'Petar Kamenečki', 'Kristijan Kopić', 'Toma Radić', 'Viktor Lukšić', 'Vito Perše', 'Karlo Krstulovic', 'Karlo Brčić', 'Ian Gluić', 'Luka Tuksar', 'Jakov Ćurić', 'Maša Dobrić', 'Marin Mihaljević', 'Marin Šego', 'Ivano Balaban', 'Jakov Jakelić', 'Lukas Balbino', 'Davor Šimić', 'Dejan Torbica', 'Frida Jakšić', 'FIlip Krznar', 'Filip Jarak', 'Annick Mareines Biščević', 'Roko Roguljić', 'Badu Pintarić', 'Božo Kelava', 'Bruno Grbac', 'Dan Poklepović', 'Taher Dahan', 'Adriana Koštomaj', 'Adrian Požega']

year_df = dfs[2]

filtered_df = year_df[year_df['volunteer_name'].isin(male_names)]

count = filtered_df['volunteer_dates'].apply(len).sum()

print(f'Prijava: {len(filtered_df)}')
print(f'Sati: {count}')

Prijava: 81
Sati: 376


In [57]:
# Number of hours general

year_df = dfs[0]

print(year_df['volunteer_dates'].apply(len).sum())

377


In [ ]:
# Number of volunteers per year

print(len(dfs[0]))

98


In [ ]:
# Number of hours

prijavljeni_volonteri = pd.DataFrame({
    'Godina': ['2022./2023.', '2023./2024.', '2024./2025.'],
    'Broj sati': [433, 624, 958]
})
prijavljeni_volonteri = prijavljeni_volonteri.sort_values(by='Broj sati', ascending=True)

plt.figure(figsize=(6, 6), dpi=800)
plt.bar(prijavljeni_volonteri['Godina'], prijavljeni_volonteri['Broj sati'], color=['skyblue', 'orange', 'green'])

# Add title and labels
plt.title('Broj sati, ukupno', fontsize=16, weight='bold', pad=20)
plt.xlabel('Broj sati', fontsize=14, labelpad=15)
plt.ylabel('Godina', fontsize=14, labelpad=15)

# Adjust layout to make room for labels
plt.tight_layout()

# Save the figure with 800 DPI before showing it
fig = plt.gcf()
fig.savefig('./REZULTATI/4_mjesec/sati_po_godinama_ukupno.png', dpi=800)

# Display the plot
plt.show()

In [ ]:
# Number of registered volunteers

prijavljeni_volonteri = pd.DataFrame({
    'Godina': ['2024./2025.', '2023./2024.', '2022./2023.'],
    'Broj volontera': [280, 127, 98]
})
prijavljeni_volonteri = prijavljeni_volonteri.sort_values(by='Broj volontera', ascending=True)

plt.figure(figsize=(6, 6), dpi=800)
plt.bar(prijavljeni_volonteri['Godina'], prijavljeni_volonteri['Broj volontera'], color=['skyblue', 'orange', 'green'])

# Add title and labels
plt.title('Broj prijavljenih po godinama', fontsize=16, weight='bold', pad=20)
plt.xlabel('Broj volontera', fontsize=14, labelpad=15)
plt.ylabel('Godina', fontsize=14, labelpad=15)

# Adjust layout to make room for labels
plt.tight_layout()

# Save the figure with 800 DPI before showing it
fig = plt.gcf()
fig.savefig('./REZULTATI/4_mjesec/prijavljenih_po_godinama.png', dpi=800)

# Display the plot
plt.show()

In [ ]:
# Per hour, all three dataframes

volunteers_per_hour1 = dfs[2].value_counts('volunteer_count').sort_index()
volunteers_per_hour2 = dfs[1].value_counts('volunteer_count').sort_index()
volunteers_per_hour3 = dfs[0].value_counts('volunteer_count').sort_index()

volunteers_per_hour1 = volunteers_per_hour1.reindex(volunteers_per_hour1.index.union(volunteers_per_hour2.index).union(volunteers_per_hour3.index), fill_value=0)
volunteers_per_hour2 = volunteers_per_hour2.reindex(volunteers_per_hour1.index, fill_value=0)
volunteers_per_hour3 = volunteers_per_hour3.reindex(volunteers_per_hour1.index, fill_value=0)

# Define width of each bar
bar_width = 0.25

# Define positions for each set of bars (the x-ticks)
x = np.arange(len(volunteers_per_hour1))

# Plot all three sets of bars
fig, ax = plt.subplots(figsize=(12, 6))

# Bar for the first dataframe
ax.bar(x - bar_width, volunteers_per_hour1, width=bar_width, color=plt.cm.Blues(np.linspace(0.4, 0.7, len(volunteers_per_hour1))), label='DataFrame 1')

# Bar for the second dataframe
ax.bar(x, volunteers_per_hour2, width=bar_width, color=plt.cm.Greens(np.linspace(0.4, 0.7, len(volunteers_per_hour2))), label='DataFrame 2')

# Bar for the third dataframe
ax.bar(x + bar_width, volunteers_per_hour3, width=bar_width, color=plt.cm.Reds(np.linspace(0.4, 0.7, len(volunteers_per_hour3))), label='DataFrame 3')

# Add title and labels
ax.set_title('Broj volontera po broju sati', fontsize=16, weight='bold', pad=20)
ax.set_xlabel('Broj sati', fontsize=14, labelpad=15)
ax.set_ylabel('Broj volontera', fontsize=14, labelpad=15)

# Set x-ticks to the positions of the bars
ax.set_xticks(x)
ax.set_xticklabels(volunteers_per_hour1.index, rotation=45)

# Add a legend
ax.legend()

# Adjust layout to make room for rotated labels
plt.tight_layout()

# Rename the items in the legend
ax.legend(['2024./2025.', '2023./2024.', '2022./2023.'])
fig.savefig('./REZULTATI/4_mjesec/volonteri_po_sati', dpi=800)

# Display the plot
plt.show()

In [ ]:
# Plot the number of volunteers per hour

year_df = dfs[2]

volunteers_per_hour = year_df['volunteer_count'].value_counts()
volunteers_per_hour = volunteers_per_hour.sort_index()

volunteers_per_hour.plot(kind='bar', figsize=(10, 6), color=plt.cm.Blues(np.linspace(0.7, 0.4, len(volunteers_per_hour))))

# Add title and labels
plt.title('Broj volontera po broju sati, 14.4.2025.', fontsize=16, weight='bold', pad=20)
plt.xlabel('Broj sati', fontsize=14, labelpad=15)
plt.ylabel('Broj volontera', fontsize=14, labelpad=15)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Adjust layout to make room for rotated labels
plt.tight_layout()  # Adjust layout to fit labels properly

# Save the figure with 800 DPI before showing it
fig = plt.gcf()  # Get the current figure
fig.savefig('./REZULTATI/4_mjesec/volonteri_po_satu-graf-24_25.png', dpi=800)

# Display the plot
plt.show()

In [ ]:
# Table for best volunteers

# Create the plot and remove axes
fig, ax = plt.subplots(figsize=(10, 4))
fig.set_dpi(800)
ax.axis('tight')
ax.axis('off')

year_df = dfs[0]

# Create the title
plt.title("Najaktivniji volonteri, 14.4.2023.", fontsize=20, weight='bold', color='black', pad=30)  # Adjust pad for less space

top_df = year_df.head(15)

# Create the table
table_data = top_df[['volunteer_name', 'volunteer_class', 'volunteer_school', 'volunteer_count']].values
table_data = table_data
column_labels = ['Ime', 'Razred', 'Škola', 'Broj sati']

# Create the table with better styling
table = ax.table(cellText=table_data, colLabels=column_labels, loc='center', cellLoc='center', colColours=['#f7f7f7']*4)

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)

# Highlight the header
for (i, j), cell in table.get_celld().items():
    if i == 0:
        cell.set_fontsize(14)
        cell.set_text_props(weight='bold')
        cell.set_facecolor('#007acc')  # Header background color
        cell.set_text_props(color='white')  # Header text color
    else:
        cell.set_edgecolor('gray')
        cell.set_facecolor('#ffffff')

# Adjust layout and display the table with reduced space
plt.subplots_adjust(left=0.1, right=0.9, top=1.5, bottom=0.2)  # Adjust the top and bottom space
plt.show()

# Save the figure
fig.savefig('./REZULTATI/4_mjesec/naj_volonteri/naj_volonteri-22_23.png', bbox_inches='tight', dpi=800)

In [ ]:
# Number of registered volunteers per school

from numpy import pad

year_df = dfs[2]

school_counts = year_df.groupby('volunteer_school').size().reset_index(name='school_count')
school_counts = school_counts.sort_values(by='school_count', ascending=False)
school_counts.columns = ['Škola', 'Broj volontera']
school_counts = school_counts.reset_index(drop=True)


# Assuming you already have 'school_counts' DataFrame
plt.figure(figsize=(10, 6), dpi=800)  # Set the DPI to 800 for both display and saving
plt.bar(school_counts['Škola'], school_counts['Broj volontera'], color=['skyblue', 'orange', 'green', 'red', 'purple'])

# Add title and labels
plt.title('Broj volontera po školama, 14.4.2025.', fontsize=16, weight='bold', pad=20)
plt.xlabel('Škola', fontsize=14, labelpad=15)
plt.ylabel('Broj volontera', fontsize=14, labelpad=15)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Adjust layout to make room for rotated labels
plt.tight_layout()  # Adjust layout to fit labels properly

# Save the figure with 800 DPI before showing it
fig = plt.gcf()  # Get the current figure
fig.savefig('./REZULTATI/4_mjesec/prijavljenih_po_skoli-graf-24_25.png', dpi=800)

# Display the plot
plt.show()

In [ ]:
year_df = dfs[2]

names_list = year_df['volunteer_name'].tolist()

female_names = []
male_names = []

for name in names_list:
    if input(name) == '':
        female_names.append(name)
    else:
        male_names.append(name)
        
print(female_names)
print(male_names)